# There are still issues with Lemmatizer part. 

I am trying to understand the reason behind it. I guess if it doesn't work, I'll need to figure out what other measures I can take to achieve this.

But I guess things would work out eventually and I just need to pay more attention.

Afterall, this is my first day onto this task and I am already nailing it so more time given, things would work out just fine!

In [1]:
import cupy
print(cupy.show_config())


OS                           : Linux-6.14.0-28-generic-x86_64-with-glibc2.39
Python Version               : 3.12.3
CuPy Version                 : 13.6.0
CuPy Platform                : NVIDIA CUDA
NumPy Version                : 2.3.2
SciPy Version                : None
Cython Build Version         : 3.0.12
Cython Runtime Version       : None
CUDA Root                    : /usr/local/cuda-12.8
nvcc PATH                    : /usr/local/cuda-12.8/bin/nvcc
CUDA Build Version           : 12090
CUDA Driver Version          : 12080
CUDA Runtime Version         : 12090 (linked to CuPy) / 12080 (locally installed)
CUDA Extra Include Dirs      : ['/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/nvidia/cuda_runtime/include']
cuBLAS Version               : (available)
cuFFT Version                : 11303
cuRAND Version               : 10309
cuSOLVER Version             : (11, 7, 3)
cuSPARSE Version             : (available)
NVRTC Version                : (12, 8)
Thrust Version    

In [2]:
# Cell 1: 
import spacy
from pathlib import Path
import os

# 查看 spaCy 版本和可用组件
spacy.info()


{'spacy_version': '3.8.7',
 'location': '/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy',
 'platform': 'Linux-6.14.0-28-generic-x86_64-with-glibc2.39',
 'python_version': '3.12.3',
 'pipelines': {}}

In [3]:
# Cell 2:
# 创建 corpus 和 models 文件夹
Path("./corpus").mkdir(parents=True, exist_ok=True)
Path("./models").mkdir(parents=True, exist_ok=True)


In [4]:
# Cell 3:
!python -m spacy convert ud_latvian/lv_lvtb-ud-train.conllu ./corpus
!python -m spacy convert ud_latvian/lv_lvtb-ud-dev.conllu ./corpus
!python -m spacy convert ud_latvian/lv_lvtb-ud-test.conllu ./corpus


ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (15055 documents):
corpus/lv_lvtb-ud-train.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2080 documents):
corpus/lv_lvtb-ud-dev.spacy
ℹ Grouping every 1 sentences into a document.
⚠ To generate better training data, you may want to group sentences
into documents with `-n 10`.
✔ Generated output file (2396 documents):
corpus/lv_lvtb-ud-test.spacy


In [5]:
# Cell 4: 初始化 config，并加入 lemmatizer
!python -m spacy init config config.cfg --lang lv --pipeline tok2vec,tagger,morphologizer,parser,lemmatizer --optimize efficiency


ℹ Generated config template specific for your use case
- Language: lv
- Pipeline: tagger, morphologizer, parser, lemmatizer
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [6]:
from pathlib import Path

# 读取原始 config.cfg
cfg_path = Path("config.cfg")
cfg_text = cfg_path.read_text(encoding="utf-8")

# 设置训练和开发语料路径
cfg_text = cfg_text.replace("train = null", "train = ./corpus/lv_lvtb-ud-train.spacy")
cfg_text = cfg_text.replace("dev = null", "dev = ./corpus/lv_lvtb-ud-dev.spacy")

# 修改 lemmatizer 为 trainable，并删除旧的 model = null 配置（如果有的话）
import re

# 删除旧的 model = null 行
cfg_text = re.sub(r'model\s*=\s*null', '', cfg_text)

# 修改 mode 为 trainable
cfg_text = re.sub(r'mode\s*=\s*["\']lookup["\']', 'mode = "trainable"', cfg_text)

# 确保 lemmatizer 的 model 下有 tok2vec 配置
lemmatizer_model_text = """
[components.lemmatizer.model]
@architectures = "spacy.Tagger.v1"
nO = null

[components.lemmatizer.model.tok2vec]
@architectures = "spacy.Tok2VecListener.v1"
upstream = "*"
width = ${components.tok2vec.model.encode.width}
"""

# 删除已有 lemmatizer.model 配置（防止重复）
cfg_text = re.sub(r'\[components\.lemmatizer\.model\][\s\S]*?(?=\[components\.)', '', cfg_text, flags=re.MULTILINE)

# 添加完整的 lemmatizer model 配置到文件末尾
cfg_text += lemmatizer_model_text

# 写回 config.cfg
cfg_path.write_text(cfg_text, encoding="utf-8")

print("✅ 已经修改 config.cfg 支持 trainable lemmatizer")


✅ 已经修改 config.cfg 支持 trainable lemmatizer


In [7]:
# Cell 6: 使用 GPU 训练
!python -m spacy train config.cfg --output ./models --paths.train ./corpus/lv_lvtb-ud-train.spacy --paths.dev ./corpus/lv_lvtb-ud-dev.spacy --gpu-id 0


ℹ Saving to output directory: models
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/typer/core.py", line 757, in main
    return _main(
           ^^^^^^
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/typer/core.py", line 195, in _main
    rv = 

In [8]:
# Cell 7: 
!python -m spacy evaluate ./models/model-best ./corpus/lv_lvtb-ud-test.spacy


ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/spacy/cli/_util.py", line 87, in setup_cli
    command(prog_name=COMMAND)
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/click/core.py", line 1442, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/typer/core.py", line 757, in main
    return _main(
           ^^^^^^
  File "/home/jesse/Projects/myenvs/spacy_lv/lib/python3.12/site-packages/typer/core.py", line 195, in _main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "/home/jesse/Project